In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:

import numpy as np
import os
import cv2
from IPython.display import clear_output

from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

In [3]:
dataset_dir = "drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/"
#work_dir = "drive/My Drive/HAM10000/"

In [4]:
list_categories = ['akiec','bcc','bkl','df','mel','nv','vasc']
num_classes = len(list_categories)
print('num_classes =', num_classes)

num_classes = 7


In [5]:
width = 32
height = width
dim = (width, height)
print(dim)

(32, 32)


TEST

In [ ]:
# TEST DATA LISTING::
list_test_filename = []
list_test_label = []

for category_no in range(len(list_categories)):
    img_dir = dataset_dir+list_categories[category_no]+'/test/'
    list_filename_single_dir = os.listdir(img_dir)
    print(img_dir, "=", len(list_filename_single_dir))

    for filename in list_filename_single_dir:
        list_test_filename.append(img_dir+filename)
        list_test_label.append(category_no)

print('len(list_test_filename) =', len(list_test_filename))
print('len(list_test_label) =', len(list_test_label))

drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/akiec/test/ = 32
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/bcc/test/ = 51
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/bkl/test/ = 109
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/df/test/ = 11
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/mel/test/ = 111
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/nv/test/ = 670
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/vasc/test/ = 14
len(list_test_filename) = 998
len(list_test_label) = 998


In [ ]:
list_test_filename[0]

'drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/akiec/test/ISIC_0024329.jpg'

In [ ]:
'''
list_test_imageset = []

#for filename in list_test_filename:
for index, filename in enumerate(list_test_filename):
    img = cv2.imread(filename, 0)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    #cv2.imwrite(save_dir+filename, resized)
    list_test_imageset.append(resized)
    print(index)
    #break

clear_output() #wait=True
print('len(list_test_imageset) =', len(list_test_imageset))
#'''

In [ ]:
def resize_and_3D(list_filenames):
    list_imageset = []

    for index, filename in enumerate(list_filenames):
        img = cv2.imread(filename, 0)
        img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        img_3d = cv2.cvtColor(img_resized,cv2.COLOR_GRAY2RGB)
        #cv2.imwrite(save_dir+filename, resized)
        list_imageset.append(img_3d)
        print(index)
        #break

    clear_output() #wait=True
    print('len(list_imageset) =', len(list_imageset))
    return list_imageset

In [ ]:
'''
np_test_imageset = np.array(list_test_imageset)
np_test_label = np.array(list_test_label)
#'''
np_test_imageset = np.array(resize_and_3D(list_test_filename))
np_test_label = np.array(list_test_label)

print('np_test_imageset.shape =', np_test_imageset.shape)
print('np_test_label.shape =', np_test_label.shape)

len(list_imageset) = 998
np_test_imageset.shape = (998, 32, 32, 3)
np_test_label.shape = (998,)


In [ ]:
np_test_norm_imageset = np_test_imageset/255.0

In [ ]:
save_filename_X = "HAM10000_Gray3D_Categorized_train_Upsampled_"+str(width)+"x"+str(height)+"_norm_test_X.pkl"
save_filename_Y = "HAM10000_Gray3D_Categorized_train_Upsampled_"+str(width)+"x"+str(height)+"_test_Y.pkl"

np.save(dataset_dir+save_filename_X, np_test_norm_imageset, allow_pickle=True)
np.save(dataset_dir+save_filename_Y, np_test_label, allow_pickle=True)

print("save_filename_X =", save_filename_X)
print("save_filename_Y =", save_filename_Y)

VALIDATION

In [ ]:
list_val_filename = []
list_val_label = []

for category_no in range(len(list_categories)):
    img_dir = dataset_dir+list_categories[category_no]+'/val/'
    list_filename_single_dir = os.listdir(img_dir)
    print(img_dir, "=", len(list_filename_single_dir))

    for filename in list_filename_single_dir:
        list_val_filename.append(img_dir+filename)
        list_val_label.append(category_no)

print('len(list_val_filename) =', len(list_val_filename))
print('len(list_val_label) =', len(list_val_label))

drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/akiec/val/ = 32
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/bcc/val/ = 51
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/bkl/val/ = 109
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/df/val/ = 11
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/mel/val/ = 111
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/nv/val/ = 670
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/vasc/val/ = 14
len(list_val_filename) = 998
len(list_val_label) = 998


In [ ]:
'''
list_val_imageset = []

#for filename in list_test_filename:
for index, filename in enumerate(list_val_filename):
    img = cv2.imread(filename, 0)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    #cv2.imwrite(save_dir+filename, resized)
    list_val_imageset.append(resized)
    print(index)
    #break

clear_output() #wait=True
print('len(list_val_imageset) =', len(list_val_imageset))
#'''

In [ ]:
np_val_imageset = np.array(resize_and_3D(list_val_filename))
np_val_label = np.array(list_val_label)

print('np_val_imageset.shape =', np_val_imageset.shape)
print('np_val_label.shape =', np_val_label.shape)

len(list_imageset) = 998
np_val_imageset.shape = (998, 32, 32, 3)
np_val_label.shape = (998,)


In [ ]:
np_val_norm_imageset = np_val_imageset/255.0

In [ ]:
save_filename_X = "HAM10000_Gray3D_Categorized_train_Upsampled_"+str(width)+"x"+str(height)+"_norm_val_X.pkl"
save_filename_Y = "HAM10000_Gray3D_Categorized_train_Upsampled_"+str(width)+"x"+str(height)+"_val_Y.pkl"

np.save(dataset_dir+save_filename_X, np_val_norm_imageset, allow_pickle=True)
np.save(dataset_dir+save_filename_Y, np_val_label, allow_pickle=True)

print("save_filename_X =", save_filename_X)
print("save_filename_Y =", save_filename_Y)

TRAINING

In [6]:
#np_imageset = np.empty((0, 450, 600), int)
list_train_filename = []
#list_train_label = []

for category in list_categories:
    img_dir = dataset_dir+category+'/train/'
    list_filename_single_dir = os.listdir(img_dir)

    list_train_filename.append(list_filename_single_dir)
    #list_train_label.append(category)

    print(img_dir, "=", len(list_filename_single_dir))

print('len(list_train_filename) =', len(list_train_filename))
#print('len(list_train_label) =', len(list_train_label))

drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/akiec/train/ = 5365
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/bcc/train/ = 5365
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/bkl/train/ = 5365
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/df/train/ = 5365
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/mel/train/ = 5365
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/nv/train/ = 5365
drive/My Drive/HAM10000/HAM10000_Gray_Categorized_train_Upsampled/vasc/train/ = 5365
len(list_train_filename) = 7


In [8]:
def my_function(list_filename):
    list_train_imageset = []
    list_train_label = []

    for image_no in range(len(list_filename[0])):
        print(image_no)

        for category_no in range(len(list_filename)):
            list_train_label.append(category_no)
            #'''
            filename = dataset_dir+list_categories[category_no]+'/train/'+list_filename[category_no][image_no]
            img = cv2.imread(filename, 0)
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            img_3d = cv2.cvtColor(resized,cv2.COLOR_GRAY2RGB)
            resized_norm = img_3d/255.0
            list_train_imageset.append(resized_norm)
            #list_train_imageset.append(img_3d)
            #'''
        #break
        #clear_output() #wait=True 

    print('len(list_train_imageset) =', len(list_train_imageset))
    print('len(list_train_label) =', len(list_train_label))
    return np.array(list_train_imageset), np.array(list_train_label)

In [ ]:
np_train_imageset, np_train_label = Parallel(n_jobs=14)(delayed(my_function)(list_train_filename) for filename in tqdm(list_train_filename))

print('np_train_imageset.shape =', np_train_imageset.shape)
print('np_train_label.shape =', np_train_label.shape)

100%|██████████| 7/7 [00:00<00:00, 595.17it/s]
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
'''
print('np_train_imageset.shape =', np_train_imageset.shape)
print('np_train_label.shape =', np_train_label.shape)
#'''

In [13]:
'''
np_train_imageset = np_train_imageset_resize_and_3D/255.0
print('np_train_imageset.shape =', np_train_imageset.shape)
#'''

"\nnp_train_imageset = np_train_imageset_resize_and_3D/255.0\nprint('np_train_imageset.shape =', np_train_imageset.shape)\n#"

In [ ]:
save_filename_X = "HAM10000_Gray3D_Categorized_train_Upsampled_"+str(width)+"x"+str(height)+"_norm_train_X.pkl"
save_filename_Y = "HAM10000_Gray3D_Categorized_train_Upsampled_"+str(width)+"x"+str(height)+"_train_Y.pkl"

np.save(dataset_dir+save_filename_X, np_train_imageset, allow_pickle=True)
np.save(dataset_dir+save_filename_Y, np_train_label, allow_pickle=True)

print("save_filename_X =", save_filename_X)
print("save_filename_Y =", save_filename_Y)